# Parallel Implementation and Evaluation of Logistic Regression
### Massively Parallel Machine Learning (2018-19)
---
#### Students: 
Marcin Paszkiewicz  
Moritz Meister

---


#### 1. Import libraries and initialize the SparkContext

In [1]:
import pyspark
import numpy as np
import pandas as pd
import time
import sys

sc = pyspark.SparkContext('local[*]')

In [2]:
sc.defaultParallelism

2

#### 2. Define helper class to read data into an RDD and simultaneously standardize it

In [3]:
class DataUtil:
    def __init__(self, sc, inputPath='data/spam.data.txt', statsInputPath='data/mean_std.txt', standardize=True):
        self.inputPath = inputPath
        self.standardize = standardize
        self.mean_std = pd.read_csv(statsInputPath, delimiter=',', header=None)
        self.mean = sc.broadcast(self.mean_std.iloc[:,0].values.astype(float))
        self.std = sc.broadcast(self.mean_std.iloc[:,1].values.astype(float))
        
        
    def read(self, sc):
        if self.standardize:
            return sc.textFile(self.inputPath).map(lambda x: np.asarray(x.split(' ')).astype(float)) \
                    .map(lambda x: (x[:56], x[57])) \
                    .map(lambda x: ((x[0] - self.mean.value[:56])/self.std.value[:56], x[1]))
        else:
            return sc.textFile(self.inputPath).map(lambda x: np.asarray(x.split(' ')).astype(float)) \
                    .map(lambda x: (x[:56], x[57]))

#### 3. Initialize the DataUtil helper and split the data into train and test set

In [4]:
spam_dataset = DataUtil(sc, 'data/spam.data.txt', 'data/mean_std.txt', True).read(sc)
train_set, test_set = spam_dataset.randomSplit(weights=[0.8, 0.2], seed=1)

print("Whole dataset: {}, {}".format(spam_dataset.count(), len(spam_dataset.first()[0])))
print("Train dataset: {}, {}".format(train_set.count(), len(train_set.first()[0])))
print("Test dataset: {}, {}".format(test_set.count(), len(test_set.first()[0])))

Whole dataset: 4601, 56
Train dataset: 3675, 56
Test dataset: 926, 56


#### 4. Functions that can't be packaged in a Python class since they need to be serializable in order to be passed to the Spark workers

In [5]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_function(y, h):
    eps = sys.float_info.epsilon
    return y * np.log(h + eps) + (1 - y) * np.log(1 - h + eps)

def stats(data):
    def f(y, pred):
        if y == pred:
            return 1,0,0,0 if y == 1 else 0,1,0,0
        elif pred == 1:
            return 0,0,1,0
        else:
            return 0,0,0,1
    tp, tn, fp, fn = data.map(lambda x: f(x[2], x[4])).reduce(lambda a, b: tuple(map(sum, zip(a, b))))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return precision, recall, f1, accuracy

#### 5. Definition of our Class containing all the functionality for the parallelized Logistic Regression

In [6]:
class ParallelLogReg():
    
    def __init__(self, data, iterations, learning_rate, lambda_reg):
        self.data = data
        self.numberObservations = self.data.count()
        self.numberFeatures = len(self.data.first()[0])
        self.iterations = iterations
        self.lr = learning_rate
        self.lambda_reg = lambda_reg
        
    def __add_intercept(self, rdd):
        return rdd.map(lambda x: (1, x[0], x[1]))
        
    def train(self, train_rdd=None, SGD=False, SGD_pct=0.5, threshold=0.5):
        eps = sys.float_info.epsilon
        
        # initialize the weights
        # w[0]: bias weight
        # w[1]: rest of the weights
        w = (0, np.zeros(self.numberFeatures))
        
        if train_rdd == None:
            data = self.__add_intercept(self.data)
        else:
            data = self.__add_intercept(train_rdd)

        # initialize prediction to rdd
        # x[0]: bias/intercept
        # x[1]: rest features
        # x[2]: true y
        # adding x[3]: predicted y
        data = data.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0])))
        numObs = data.count()
        
        train = data.cache()
        
        for i in range(self.iterations):
            start = time.time()
            
            # sample for SGD
            if SGD:
                train = data.sample(False, SGD_pct).repartition(sc.defaultParallelism).cache()
                if i == 0:
                    numObs = train.count()
            
            # compute derivatives
            temp = train.map(lambda x: ((x[3] - x[2]) * x[0], (x[3] - x[2]) * x[1])) \
                       .reduce(lambda a,b: (a[0] + b[0], a[1] + b[1]))
            dw = (temp[0]/numObs, (temp[1]/numObs) + self.lambda_reg * w[1])
            
            # update weights
            w = (w[0] - self.lr * dw[0], w[1] - self.lr * dw[1])
            
            # update prediction
            train = train.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0]))).cache()
            
            if (i % 5 == 0):
                end = time.time()
                
                model_stats = self.validate(train, w, add_intercept=False)

                print("Iteration: " + str(i) + ", Total Time: " + str(end - start))
                print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(model_stats['precision'], model_stats['recall'], model_stats['f1'], model_stats['accuracy']))
                print('Current loss: {}\n'.format(model_stats['loss']))
                
        model_stats = self.validate(data, w, add_intercept=False)
        
        print('Training stats:')
        print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(model_stats['precision'], model_stats['recall'], model_stats['f1'], model_stats['accuracy']))
        print('Final training loss: {}\n'.format(model_stats['loss']))
            
        return {**model_stats, 'w': w}
    
    def validate(self, val_rdd, w, threshold=0.5, add_intercept=True):
        if add_intercept:
            val_rdd = self.__add_intercept(val_rdd)
        
        numObs = val_rdd.count()
        
        # update prediction
        val_rdd = val_rdd.map(lambda x: (x[0], x[1], x[2], sigmoid(w[1].dot(x[1]) + w[0] * x[0])))
        
        loss = val_rdd.map(lambda x: cost_function(x[2], x[3])) \
                    .reduce(lambda a,b: a + b)
        loss = -(1/numObs) * loss + (self.lambda_reg/2) * np.sum(w[1]**2)
        
        # calculate stats
        val_rdd = val_rdd.map(lambda x: (x[0], x[1], x[2], x[3], 1 if x[3] >= threshold else 0)).cache()
        precision, recall, f1, accuracy = stats(val_rdd)
        return {'loss': loss, 'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy}
    
    def cross_validate(self, k, SGD=False, SGD_pct = 0.5):
        
        if k == 1:
            print("Please choose a k > 1. Or use regular train function")
            return
        
        # even if it is ordered, this procedure should produce folds with similar distributions
        dataWithIndex = self.data.zipWithIndex();
        
        model_stats = []
        models = []
        
        for i in range(k):
            print("Fold: " + str(i))
            
            fold = dataWithIndex.filter(lambda x: x[1] % k != i).repartition(sc.defaultParallelism) \
                                .map(lambda x: x[0])
            
            model = self.train(fold, SGD, SGD_pct)
            models.append(model)
            
            val = dataWithIndex.filter(lambda x: x[1] % k == i).repartition(sc.defaultParallelism) \
                                .map(lambda x: x[0])
            
            val_stats = self.validate(val, model['w'])
            model_stats.append(val_stats)
            print('Validation stats:')
            print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(val_stats['precision'], val_stats['recall'], val_stats['f1'], val_stats['accuracy']))
            print('Loss: {}\n'.format(val_stats['loss']))

        avg_model_stats = {}
        for key in model_stats[0].keys():
            avg_model_stats['avg_' + key] = sum(stat[key] for stat in model_stats) / len(model_stats)

        print('Averaged validation stats:')
        print('Precision: {}, Recall: {}, F1: {}, Accuracy: {}'.format(avg_model_stats['avg_precision'], avg_model_stats['avg_recall'], avg_model_stats['avg_f1'], avg_model_stats['avg_accuracy']))
        print('Loss: {}'.format(avg_model_stats['avg_loss']))

#### 6. Initialize a new Logistic Regression

In [7]:
logReg = ParallelLogReg(data=train_set, iterations=200, learning_rate=0.1, lambda_reg=0.05)

#### 7. Perform Cross-Validation with the above specified parameters and 10 folds

In [ ]:
start = time.time()
logReg.cross_validate(10)
print('Total time:')
print(time.time() - start)

#### 8. Train the model with 80% train set and the previously found best hyperparameters

In [8]:
model = logReg.train(train_set)

Iteration: 0, Total Time: 0.9133286476135254
Precision: 0.9399373754625676, Recall: 0.9532332563510393, F1: 0.9465386269170131, Accuracy: 0.8985034013605442
Current loss: 0.6492381078517572

Iteration: 5, Total Time: 0.22257184982299805
Precision: 0.9488425264361247, Recall: 0.9496567505720824, F1: 0.9492494639027877, Accuracy: 0.9034013605442177
Current loss: 0.5214280133698086

Iteration: 10, Total Time: 0.23688960075378418
Precision: 0.9535417264123889, Recall: 0.946484486194136, F1: 0.95, Accuracy: 0.9047619047619048
Current loss: 0.4610221795060205

Iteration: 15, Total Time: 0.22551560401916504
Precision: 0.9548331415420023, Recall: 0.9434337691870381, F1: 0.9490992279096367, Accuracy: 0.9031292517006803
Current loss: 0.4264075428524229

Iteration: 20, Total Time: 0.1652381420135498
Precision: 0.9574100719424461, Recall: 0.9432945846328324, F1: 0.9502999143101972, Accuracy: 0.9053061224489796
Current loss: 0.40425820021823244

Iteration: 25, Total Time: 0.22816133499145508
Precis

Validate the model on the 20% test set with 0.5 threshold:

In [10]:
logReg.validate(test_set, model['w'], threshold=0.5)

{'loss': 0.3917613943399057,
 'precision': 0.9543325526932084,
 'recall': 0.9188275084554679,
 'f1': 0.9362435381964388,
 'accuracy': 0.8801295896328294}

Appendix: how to compute the precision/recall curve

In [ ]:
for i in range(21):
    model_stats = logReg.validate(test_set, model['w'], threshold=i/20)
    print('{}'.format(model_stats['recall']))
